In [1]:
import numpy as np
import pandas as pd
import os
import io
import seaborn as sns
import requests
import json

import matplotlib.pyplot as plt
import pymc3 as pm

import scipy
import scipy.stats as stats
import scipy.optimize as opt
#import statsmodels.api as sm

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
plt.style.use('seaborn-darkgrid')
pd.set_option('display.max_columns', 500)
%matplotlib inline 

In [2]:
#Get NHL Data for Last Season
r = requests.get('https://statsapi.web.nhl.com/api/v1/schedule?stats=gameLog&season=20182019')
data = r.json()

In [3]:
data = pd.read_csv("combined_hockey2.csv")

In [4]:
data.columns

Index(['Unnamed: 0', 'Assist1', 'Assist1_ID', 'Assist2', 'Assist2_ID',
       'Assists2_ID', 'Away', 'Away_Score', 'EmptyNet', 'Event', 'EventID',
       'Event_Description', 'Event_Team', 'Event_Type_ID', 'Final_Away_Score',
       'Final_Home_Score', 'GameDate', 'GameID', 'GameState', 'GameType',
       'Goalie', 'Goalie_ID', 'Home', 'Home_Score', 'Period', 'Period_Type',
       'Player_1_ID', 'Player_1_Name', 'Player_2_ID', 'Player_2_Name',
       'Scorer', 'Scorer_ID', 'Shooter', 'Shooter_ID', 'ShotType',
       'Time_Remaining', 'Time_in_Period', 'Venue', 'x', 'y', 'Event2', 'Side',
       'Season', 'Attacking', 'y_stdz', 'x_stdz', 'y_normal', 'x_normal',
       'Event_Team_Adj', 'y_same', 'x_same', 'zone_x', 'zone_y', 'shot_zone',
       'Opposition Team'],
      dtype='object')

In [5]:
players = data[['Player_1_Name', 'Player_1_ID', 'Season']].groupby(['Player_1_Name', 'Player_1_ID', 'Season']).count().reset_index()

In [6]:
players

,Player_1_Name,Player_1_ID,Season
0,A.J. Greer,8478421,2017-2018
1,A.J. Greer,8478421,2018-2019
2,Aaron Dell,8477180,2017-2018
3,Aaron Dell,8477180,2018-2019
4,Aaron Ekblad,8477932,2017-2018
5,Aaron Ekblad,8477932,2018-2019
6,Aaron Ness,8474604,2017-2018
7,Aaron Ness,8474604,2018-2019
8,Adam Boqvist,8480871,2018-2019
9,Adam Brooks,8478996,2018-2019


In [7]:
#Get NHL Data for Last Season
r = requests.get('https://statsapi.web.nhl.com/api/v1/people/8478421?season=20182019')
data = r.json()

In [8]:
data

{'copyright': 'NHL and the NHL Shield are registered trademarks of the National Hockey League. NHL and NHL team marks are the property of the NHL and its teams. © NHL 2018. All Rights Reserved.',
 'people': [{'id': 8478421,
   'fullName': 'A.J. Greer',
   'link': '/api/v1/people/8478421',
   'firstName': 'A.J.',
   'lastName': 'Greer',
   'primaryNumber': '24',
   'birthDate': '1996-12-14',
   'currentAge': 21,
   'birthCity': 'Joliette',
   'birthStateProvince': 'QC',
   'birthCountry': 'CAN',
   'nationality': 'CAN',
   'height': '6\' 3"',
   'weight': 210,
   'active': True,
   'alternateCaptain': False,
   'captain': False,
   'rookie': True,
   'shootsCatches': 'L',
   'rosterStatus': 'N',
   'currentTeam': {'id': 21,
    'name': 'Colorado Avalanche',
    'link': '/api/v1/teams/21'},
   'primaryPosition': {'code': 'L',
    'name': 'Left Wing',
    'type': 'Forward',
    'abbreviation': 'LW'}}]}

In [31]:
#Player Vitals for 2017-2018
name = []
primaryNumber = []
birthDate = []
#currentAge = []
birthCity = []
birthStateProvince = []
birthCountry = []
nationality = []
height = []
weight = []
active = []
alternateCaptain = []
captain = []
rookie = []
shootsCatches = []
rosterStatus = []
currentTeam = []
primaryPosition = []

for player in players[players['Season'] == '2017-2018']['Player_1_ID']:
    if player != 'None':
        r = requests.get('https://statsapi.web.nhl.com/api/v1/people/'+ player + '?season=20172018')
        data = r.json()
        data = data['people'][0]
        
        try:
            name.append(data['fullName'])
        except:
            name.append('NA')
        try:
            primaryNumber.append(data['primaryNumber'])
        except:
            primaryNumber.append('NA')
        try:
            birthDate.append(data['birthDate'])
        except:
            birthDate.append('NA')
        #currentAge.append(data['currentAge'])
        try:
            birthCity.append(data['birthCity'])
        except:
            birthCity.append('NA')
        try:
            birthStateProvince.append(data['birthStateProvince'])
        except:
            birthStateProvince.append('NA')
        try:
            birthCountry.append(data['birthCountry'])
        except:
            birthCountry.append('NA')
        try:
            nationality.append(data['nationality'])
        except:
            nationality.append('NA')
        try:
            height.append(data['height'])
        except:
            height.append('NA')
        try:
            weight.append(data['weight'])
        except:
            weight.append('NA')
        try:
            active.append(data['active'])
        except:
            active.append('NA')
        try:
            alternateCaptain.append(data['alternateCaptain'])
        except:
            alternateCaptain.append('NA')
        try:
            captain.append(data['captain'])
        except:
            captain.append('NA')
        try:
            rookie.append(data['rookie'])
        except:
            rookie.append('NA')
        try:
            shootsCatches.append(data['shootsCatches'])
        except:
            shootsCatches.append('NA')
        try:
            rosterStatus.append(data['rosterStatus'])
        except:
            rosterStatus.append('NA')
        try:
            currentTeam.append(data['currentTeam']['name'])
        except:
            currentTeam.append('NA')
        try:
            primaryPosition.append(data['primaryPosition']['abbreviation'])
        except:
            primaryPosition.append('NA')

In [34]:
player_vitals_20172018 = pd.DataFrame(
    {'Name': name, 'PrimaryNumber': primaryNumber, 'BirthDate': birthDate, 'BirthCity': birthCity,
    'BirthState': birthStateProvince, 'BirthCountry': birthCountry, 'Nationality': nationality,
    'Height': height, 'Weight': weight, 'Active': active, 'AlternateCaptain':alternateCaptain,
    'Captain': captain, 'Rookie': rookie, 'Shoots_Catches': shootsCatches, 'RosterStatus': rosterStatus,
    'CurrentTeam': currentTeam, 'Position': primaryPosition})
player_vitals_20172018['Season'] = '2017-2018'

In [36]:
player_vitals_20172018.to_csv("player_vitals_20172018.csv", index=False)

In [37]:
#Player Vitals for 2018-2019
name = []
primaryNumber = []
birthDate = []
#currentAge = []
birthCity = []
birthStateProvince = []
birthCountry = []
nationality = []
height = []
weight = []
active = []
alternateCaptain = []
captain = []
rookie = []
shootsCatches = []
rosterStatus = []
currentTeam = []
primaryPosition = []

for player in players[players['Season'] == '2018-2019']['Player_1_ID']:
    if player != 'None':
        r = requests.get('https://statsapi.web.nhl.com/api/v1/people/'+ player + '?season=20182019')
        data = r.json()
        data = data['people'][0]
        
        try:
            name.append(data['fullName'])
        except:
            name.append('NA')
        try:
            primaryNumber.append(data['primaryNumber'])
        except:
            primaryNumber.append('NA')
        try:
            birthDate.append(data['birthDate'])
        except:
            birthDate.append('NA')
        #currentAge.append(data['currentAge'])
        try:
            birthCity.append(data['birthCity'])
        except:
            birthCity.append('NA')
        try:
            birthStateProvince.append(data['birthStateProvince'])
        except:
            birthStateProvince.append('NA')
        try:
            birthCountry.append(data['birthCountry'])
        except:
            birthCountry.append('NA')
        try:
            nationality.append(data['nationality'])
        except:
            nationality.append('NA')
        try:
            height.append(data['height'])
        except:
            height.append('NA')
        try:
            weight.append(data['weight'])
        except:
            weight.append('NA')
        try:
            active.append(data['active'])
        except:
            active.append('NA')
        try:
            alternateCaptain.append(data['alternateCaptain'])
        except:
            alternateCaptain.append('NA')
        try:
            captain.append(data['captain'])
        except:
            captain.append('NA')
        try:
            rookie.append(data['rookie'])
        except:
            rookie.append('NA')
        try:
            shootsCatches.append(data['shootsCatches'])
        except:
            shootsCatches.append('NA')
        try:
            rosterStatus.append(data['rosterStatus'])
        except:
            rosterStatus.append('NA')
        try:
            currentTeam.append(data['currentTeam']['name'])
        except:
            currentTeam.append('NA')
        try:
            primaryPosition.append(data['primaryPosition']['abbreviation'])
        except:
            primaryPosition.append('NA')

In [40]:
player_vitals_20182019 = pd.DataFrame(
    {'Name': name, 'PrimaryNumber': primaryNumber, 'BirthDate': birthDate, 'BirthCity': birthCity,
    'BirthState': birthStateProvince, 'BirthCountry': birthCountry, 'Nationality': nationality,
    'Height': height, 'Weight': weight, 'Active': active, 'AlternateCaptain': alternateCaptain,
    'Captain': captain, 'Rookie': rookie, 'Shoots_Catches': shootsCatches, 'RosterStatus': rosterStatus,
    'CurrentTeam': currentTeam, 'Position': primaryPosition})
player_vitals_20182019['Season'] = '2018-2019'

In [41]:
player_vitals_20182019.to_csv("player_vitals_20182019.csv", index=False)

In [96]:
###Now we need some stats!
r = requests.get('https://statsapi.web.nhl.com/api/v1/people/'+ '8478421/stats' + '?stats=statsSingleSeason&season=20182019')
data = r.json()

In [100]:
stats_dict = {}
for stat in data['stats'][0]['splits'][0]['stat']:
    stats_dict[stat] = []
stats_dict['PlayerID'] = []

In [101]:
for player in players[players['Season'] == '2017-2018']['Player_1_ID']:
    ###Now we need some stats!
    r = requests.get('https://statsapi.web.nhl.com/api/v1/people/'+ player + '/stats?stats=statsSingleSeason&season=20172018')
    data = r.json()
    
    for key in stats_dict.keys():
        if key == 'PlayerID':
            stats_dict[key].append(player)
        else:
            try:
                stats_dict[key].append(data['stats'][0]['splits'][0]['stat'][key])
            except:
                stats_dict[key].append('NA')

In [108]:
player_stats_20172018_shooters = pd.DataFrame(stats_dict)

In [109]:
player_stats_20172018_shooters = player_stats_20172018.merge(players[['Player_1_ID', 'Player_1_Name']].drop_duplicates(),
                                                   left_on=['PlayerID'], right_on=['Player_1_ID'],
                                                   how='left')

In [110]:
player_stats_20172018_shooters.to_csv("player_stats_20172018_shooters.csv", index=False)

In [111]:
###Now we need some stats!
r = requests.get('https://statsapi.web.nhl.com/api/v1/people/'+ '8478421/stats' + '?stats=statsSingleSeason&season=20182019')
data = r.json()
stats_dict = {}
for stat in data['stats'][0]['splits'][0]['stat']:
    stats_dict[stat] = []
stats_dict['PlayerID'] = []

In [112]:
for player in players[players['Season'] == '2018-2019']['Player_1_ID']:
    ###Now we need some stats!
    r = requests.get('https://statsapi.web.nhl.com/api/v1/people/'+ player + '/stats?stats=statsSingleSeason&season=20182019')
    data = r.json()
    
    for key in stats_dict.keys():
        if key == 'PlayerID':
            stats_dict[key].append(player)
        else:
            try:
                stats_dict[key].append(data['stats'][0]['splits'][0]['stat'][key])
            except:
                stats_dict[key].append('NA')

In [113]:
player_stats_20182019_shooters = pd.DataFrame(stats_dict)
player_stats_20182019_shooters = player_stats_20182019_shooters.merge(players[['Player_1_ID', 'Player_1_Name']].drop_duplicates(),
                                                   left_on=['PlayerID'], right_on=['Player_1_ID'],
                                                   how='left')

In [114]:
player_stats_20182019_shooters.to_csv("player_stats_20182019_shooters.csv", index=False)

In [ ]:
#Now We Need Goalie Stats! 


In [84]:
# Aaron Dell = 8477180

In [102]:
len(stats_dict['games'])

1386

In [103]:
len(players[players['Season'] == '2017-2018'])

1386